In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
from sklearn.metrics import classification_report,confusion_matrix

# Path to image folders, 
data_path = fr'../Datasets/full/Training'
#data_path = fr'../Datasets/config'

def remove_white_background(pixels):
    newPixels = []
    for pixel in pixels:
        pixel = list(pixel)
        if ((256 > pixel[0] > 200) and (256 > pixel[1] > 200) and (256 > pixel[2] > 200)):
            pixel[0] = 0
            pixel[1] = 0
            pixel[2] = 0
        newPixels.append(pixel)
    
    return newPixels


def redify(pixels):
    return [r for r, g, b in pixels]

                
def greenify(pixels):
    return [g for r, g, b in pixels]


def blueify(pixels):
    return [b for r, g, b in pixels]


def get_rgb_pixels_onehot_labels(src):
    print("Starting...")
    newPixels = []
    y = np.empty(shape=[0, 1])

    for subdir in os.listdir(src):
        current_path = os.path.join(src, subdir)
        for file in os.listdir(current_path):
            img = Image.open(os.path.join(current_path, file))
            imgResize = img.resize((24,24))
            pixels = list(imgResize.getdata())
            pixels = remove_white_background(pixels)
            newPixels.append(pixels)
            y = np.append(y, subdir)
    return newPixels, LabelBinarizer().fit_transform(y) # OneHot encode y


def process_files(src):
    X_red_train = []
    X_red_validation = []
    X_red_test = []
    X_green_train = []
    X_green_validation = []
    X_green_test = []
    X_blue_train = []
    X_blue_validation = []
    X_blue_test = []
    all_pixels, y = get_rgb_pixels_onehot_labels(src)

    X_train, X_validation, y_train, y_validation = train_test_split(all_pixels, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)
    
    for pixels in X_train:       
        X_red_train.append(redify(pixels.copy()))
        X_green_train.append(greenify(pixels.copy()))
        X_blue_train.append(blueify(pixels.copy()))
        
    for pixels in X_validation:       
        X_red_validation.append(redify(pixels.copy()))
        X_green_validation.append(greenify(pixels.copy()))
        X_blue_validation.append(blueify(pixels.copy()))
        
    for pixels in X_test:       
        X_red_test.append(redify(pixels.copy()))
        X_green_test.append(greenify(pixels.copy()))
        X_blue_test.append(blueify(pixels.copy()))
    
    
    print("Finished \n")
    return np.asarray(X_red_train), np.asarray(X_red_validation), np.asarray(X_red_test), np.asarray(X_green_train), np.asarray(X_green_validation), np.asarray(X_green_test), np.asarray(X_blue_train), np.asarray(X_blue_validation), np.asarray(X_blue_test), y_train, y_validation, y_test


def get_youdens_index(predictions, Y):
    # Calculate true positive/negative and false positive/negative
    tp = sum((Y == predictions) * (Y == 1) * 1)
    tn = sum((Y == predictions) * (Y == 0) * 1)
    fp = sum((Y != predictions) * (Y == 0) * 1)
    fn = sum((Y != predictions) * (Y == 1) * 1)
    
    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (fp + tn)
    
    result = sensitivity - (1 - specificity)
    # Put it in a dateframe for nicer visuals
    df = pd.DataFrame({'Youdens Index': result})
    pd.set_option('display.max_rows', 200)
    
    return df

In [2]:
# Call process_files and assign variables
X_red_train, X_red_validation, X_red_test, X_green_train, X_green_validation, X_green_test, X_blue_train, X_blue_validation, X_blue_test, Y_train, Y_validation, Y_test = process_files(data_path)

Starting...
Finished 



In [3]:
import time
from sklearn import svm
from sklearn.neural_network import MLPClassifier

# Red Config HL Test
print("Red Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, random_state=1)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")

# Green Config HL Test
print("Green Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, random_state=1)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
# Blue Config HL Test
print("Blue Config HL Tests")
for hl in [(432), (216), (216, 108), (108), (108, 54)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, random_state=1)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")

Red Config HL Tests
Hidden layers:  432
Score:  0.9690721649484536 Time:  2.0528647899627686
   Youdens Index
0       0.933333
1       0.966692
2       0.987829
3       1.000000
4       1.000000
5       0.983908 

Hidden layers:  216
Score:  0.9656357388316151 Time:  1.1680610179901123
   Youdens Index
0       0.998028
1       0.968750
2       0.957526
3       0.986301
4       1.000000
5       0.972414 

Hidden layers:  (216, 108)
Score:  0.9690721649484536 Time:  1.1930835247039795
   Youdens Index
0       0.984694
1       0.968750
2       0.983688
3       0.996071
4       1.000000
5       0.974900 

Hidden layers:  108
Score:  0.9707903780068728 Time:  0.8277521133422852
   Youdens Index
0       0.971361
1       0.991770
2       0.979798
3       0.986301
4       1.000000
5       0.979404 

Hidden layers:  (108, 54)
Score:  0.9707903780068728 Time:  1.0549585819244385
   Youdens Index
0       0.973333
1       0.914609
2       0.989899
3       0.986301
4       1.000000
5       0.979498

In [4]:
# default alpha=0.0001

print("Red Config Alpha Tests")
# Red Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=(216, 108), alpha=a, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_red_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_red_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_red_validation), Y_validation), "\n")
    
print("Green Config Alpha Tests")
# Green Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=(216, 108), alpha=a, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_green_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_green_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_green_validation), Y_validation), "\n")
    
print("Blue Config Alpha Tests")
# Blue Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=(216, 108), alpha=a, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_blue_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_blue_validation, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_blue_validation), Y_validation), "\n")    

Red Config Alpha Tests
Alpha:  0.0001
Score:  0.9690721649484536 Time:  1.148042917251587
   Youdens Index
0       0.984694
1       0.968750
2       0.983688
3       0.996071
4       1.000000
5       0.974900 

Alpha:  0.001
Score:  0.9742268041237113 Time:  1.2531378269195557
   Youdens Index
0       0.986667
1       0.968750
2       0.983688
3       0.998035
4       1.000000
5       0.977199 

Alpha:  0.01
Score:  0.9759450171821306 Time:  1.1350305080413818
   Youdens Index
0       0.986667
1       0.977109
2       0.985758
3       0.996071
4       1.000000
5       0.977199 

Alpha:  0.1
Score:  0.9725085910652921 Time:  1.477342128753662
   Youdens Index
0       0.986667
1       0.977109
2       0.983688
3       0.996071
4       1.000000
5       0.972601 

Alpha:  1


c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Score:  0.9879725085910653 Time:  11.98595929145813
   Youdens Index
0       0.984694
1       0.968750
2       0.997930
3       1.000000
4       1.000000
5       0.997701 

Green Config Alpha Tests
Alpha:  0.0001
Score:  0.9896907216494846 Time:  0.6616010665893555
   Youdens Index
0       0.956055
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       0.997701 

Alpha:  0.001
Score:  0.9896907216494846 Time:  0.5775246620178223
   Youdens Index
0       0.958028
1       1.000000
2       0.989899
3       0.998035
4       1.000000
5       1.000000 

Alpha:  0.01
Score:  0.9914089347079038 Time:  0.7316653728485107
   Youdens Index
0       0.969389
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       0.997701 

Alpha:  0.1
Score:  0.9896907216494846 Time:  0.7781603336334229
   Youdens Index
0       0.958028
1       1.000000
2       1.000000
3       0.998035
4       1.000000
5       0.997701 

Alpha:  1


c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Score:  1.0 Time:  12.456931591033936
   Youdens Index
0            1.0
1            1.0
2            1.0
3            1.0
4            1.0
5            1.0 

Blue Config Alpha Tests
Alpha:  0.0001
Score:  0.9828178694158075 Time:  0.5374882221221924
   Youdens Index
0       0.946667
1       0.997942
2       0.979547
3       1.000000
4       1.000000
5       1.000000 

Alpha:  0.001
Score:  0.9810996563573883 Time:  0.5613760948181152
   Youdens Index
0       0.946667
1       0.997942
2       0.979547
3       0.986301
4       1.000000
5       1.000000 

Alpha:  0.01
Score:  0.9828178694158075 Time:  0.5615098476409912
   Youdens Index
0       0.946667
1       0.997942
2       0.989648
3       0.986301
4       1.000000
5       1.000000 

Alpha:  0.1
Score:  0.9828178694158075 Time:  0.5875339508056641
   Youdens Index
0       0.960000
1       0.997942
2       0.979547
3       0.986301
4       1.000000
5       1.000000 

Alpha:  1
Score:  1.0 Time:  12.543936967849731
   Youdens Index
0 

c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
# Red
mlpc_red =  MLPClassifier(hidden_layer_sizes=((216, 108)), alpha=0.1, activation='relu', solver='adam', random_state=1, max_iter=500)
mlpc_red.fit(X_red_train, Y_train)
vector_red = mlpc_red.predict(X_red_train)

# Green
mlpc_green = MLPClassifier(hidden_layer_sizes=((216, 108)), alpha=0.1, activation='relu', solver='adam', random_state=1, max_iter=500)
mlpc_green.fit(X_green_train, Y_train)
vector_green = mlpc_green.predict(X_green_train)

# Blue
mlpc_blue = MLPClassifier(hidden_layer_sizes=((216, 108)), alpha=0.1, activation='relu', solver='adam', random_state=1, max_iter=500)
mlpc_blue.fit(X_blue_train, Y_train)
vector_blue = mlpc_blue.predict(X_blue_train)

# Validation
vector_red_val = mlpc_red.predict(X_red_validation)
vector_green_val = mlpc_green.predict(X_green_validation)
vector_blue_val = mlpc_blue.predict(X_blue_validation)


print("Red Model Validation")
print(classification_report(Y_validation, vector_red_val, zero_division=0))
print(get_youdens_index(vector_red_val, Y_validation), "\n")

print("Green Model Validation")
print(classification_report(Y_validation, vector_green_val, zero_division=0))
print(get_youdens_index(vector_green_val, Y_validation), "\n")

print("Blue Model Validation")
print(classification_report(Y_validation, vector_blue_val, zero_division=0))
print(get_youdens_index(vector_blue_val, Y_validation), "\n")

Red Model Validation
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        75
           1       0.99      0.98      0.98        96
           2       0.97      0.99      0.98        99
           3       0.97      1.00      0.99        73
           4       1.00      1.00      1.00        92
           5       0.96      0.99      0.97       147

   micro avg       0.98      0.99      0.98       582
   macro avg       0.98      0.99      0.99       582
weighted avg       0.98      0.99      0.98       582
 samples avg       0.98      0.99      0.98       582

   Youdens Index
0       0.986667
1       0.977109
2       0.983688
3       0.996071
4       1.000000
5       0.972601 

Green Model Validation
              precision    recall  f1-score   support

           0       0.99      0.96      0.97        75
           1       1.00      1.00      1.00        96
           2       1.00      1.00      1.00        99
           3       0.

In [6]:
# Stack the rgb predictions to get combi model values 
X_combined_train = np.column_stack((vector_red, vector_green, vector_blue)) # shape (1745, 18) - 18 features
X_combined_val = np.column_stack((vector_red_val, vector_green_val, vector_blue_val)) # shape (582, 18) - 18 features

# Combi Config HL Test
print("Combi Config HL Tests")
for hl in [(15), (8), (8, 4)]:
    print("Hidden layers: ", hl)
    mlp = MLPClassifier(hidden_layer_sizes=hl, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_combined_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_combined_val, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_combined_val), Y_validation), "\n")


print("Combi Config Alpha Tests")
# Combi Model ALpha Config
for a in [0.0001, 0.001, 0.01, 0.1, 1]:
    print("Alpha: ", a)
    mlp = MLPClassifier(hidden_layer_sizes=15, alpha=a, max_iter=500, random_state=1)
    t0 = time.time()
    mlp.fit(X_combined_train, Y_train)
    t1 = time.time()
    print("Score: ", mlp.score(X_combined_val, Y_validation), "Time: ", t1 - t0)
    print(get_youdens_index(mlp.predict(X_combined_val), Y_validation), "\n")

Combi Config HL Tests
Hidden layers:  15
Score:  0.9982817869415808 Time:  0.4855995178222656
   Youdens Index
0       0.986667
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       1.000000 

Hidden layers:  8
Score:  0.9982817869415808 Time:  0.634249210357666
   Youdens Index
0       0.986667
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       1.000000 

Hidden layers:  (8, 4)


c:\users\olive\documents\6. semester\applied artificial intelligence\ai_exam\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Score:  0.15807560137457044 Time:  1.1906373500823975
   Youdens Index
0       0.000000
1       0.826646
2       0.000000
3       0.000000
4       0.000000
5       0.000000 

Combi Config Alpha Tests
Alpha:  0.0001
Score:  0.9982817869415808 Time:  0.4880216121673584
   Youdens Index
0       0.986667
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       1.000000 

Alpha:  0.001
Score:  0.9982817869415808 Time:  0.4960317611694336
   Youdens Index
0       0.986667
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       1.000000 

Alpha:  0.01
Score:  0.9982817869415808 Time:  0.4814479351043701
   Youdens Index
0       0.986667
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       1.000000 

Alpha:  0.1
Score:  0.9982817869415808 Time:  0.4504094123840332
   Youdens Index
0       0.986667
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       1.000000 

Alpha:  1
Score:  0.9948453608247423 Time:  0.6816

In [8]:
# Combi Train
mlpc_combi =  MLPClassifier(hidden_layer_sizes=(15), alpha=0.01, activation='relu', solver='adam', random_state=1, max_iter=300)
mlpc_combi.fit(X_combined_train, Y_train)

# Validation
print("Combined Model Validation")
combi_val = mlpc_combi.predict(X_combined_val)
print(classification_report(Y_validation, combi_val, zero_division=0))
print(get_youdens_index(combi_val, Y_validation), "\n")

Combined Model Validation
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        75
           1       1.00      1.00      1.00        96
           2       1.00      1.00      1.00        99
           3       1.00      1.00      1.00        73
           4       1.00      1.00      1.00        92
           5       1.00      1.00      1.00       147

   micro avg       1.00      1.00      1.00       582
   macro avg       1.00      1.00      1.00       582
weighted avg       1.00      1.00      1.00       582
 samples avg       1.00      1.00      1.00       582

   Youdens Index
0       0.986667
1       1.000000
2       1.000000
3       1.000000
4       1.000000
5       1.000000 



In [ ]:
vector_red_test = mlpc_red.predict(X_red_test)
vector_green_test = mlpc_green.predict(X_green_test)
vector_blue_test = mlpc_blue.predict(X_blue_test)

X_combined_test = np.column_stack((vector_red_test, vector_green_test, vector_blue_test)) # shape (582, 18) - 18 features

# Test Final
print("Combined Model Test")
combi_test = mlpc_combi.predict(X_combined_test)
print(classification_report(Y_test, combi_test, zero_division=0))
print(get_youdens_index(combi_test, Y_test), "\n")